# 🚀 Trading Cuantitativo con ETFs - Notebook Inicial

Este notebook muestra:
1. Descarga de datos de ETFs con `yfinance`.
2. Cálculo de retornos y métricas básicas.
3. Visualización de precios y retornos.
4. Backtest simple (Buy & Hold y cruce de medias).


In [ ]:
# Instalación de librerías necesarias
!pip install yfinance backtrader empyrical

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import empyrical as ep
import backtrader as bt

In [ ]:
# Descargar datos de ETFs (SPY = S&P 500, TLT = Bonos largo plazo, QQQ = Nasdaq)
tickers = ["SPY", "TLT", "QQQ"]
data = yf.download(tickers, start="2015-01-01", end="2025-01-01")["Adj Close"]
data.head()

In [ ]:
# Calcular rendimientos
returns = data.pct_change().dropna()
log_returns = (1 + returns).apply(np.log)

# Gráfico de precios
data.plot(figsize=(12,6), title="Precios ETFs (2015-2025)")
plt.show()

# Retornos acumulados
(1 + returns).cumprod().plot(figsize=(12,6), title="Retornos Acumulados")
plt.show()

In [ ]:
# Métricas básicas con empyrical (ejemplo SPY)
spy = returns["SPY"]
print("📊 Métricas SPY:")
print("Retorno anualizado:", ep.annual_return(spy))
print("Volatilidad anualizada:", ep.annual_volatility(spy))
print("Sharpe Ratio:", ep.sharpe_ratio(spy))
print("Max Drawdown:", ep.max_drawdown(spy))

In [ ]:
# Backtest simple: Cruce de medias con Backtrader
class SmaCross(bt.Strategy):
    params = dict(pfast=20, pslow=50)

    def __init__(self):
        sma1 = bt.ind.SMA(period=self.p.pfast)
        sma2 = bt.ind.SMA(period=self.p.pslow)
        self.crossover = bt.ind.CrossOver(sma1, sma2)

    def next(self):
        if not self.position:
            if self.crossover > 0:
                self.buy()
        elif self.crossover < 0:
            self.close()

cerebro = bt.Cerebro()
cerebro.addstrategy(SmaCross)

spy_data = bt.feeds.PandasData(dataname=yf.download("SPY", start="2015-01-01", end="2025-01-01"))
cerebro.adddata(spy_data)
cerebro.broker.setcash(10000)

cerebro.run()
cerebro.plot()